In [1]:
import getpass
import os

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

/Users/jadhavjaichandra/Desktop/IP_LINK_AI/IP_LINK_AI_ENV/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# import time

# index_name = "ip-link2"

# existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

# if index_name not in existing_indexes:
#     pc.create_index(
#         name=index_name,
#         dimension=768,
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#     )
#     while not pc.describe_index(index_name).status["ready"]:
#         time.sleep(1)

# index = pc.Index(index_name)

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [6]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
# import re
# from urllib.parse import unquote

# def clean_markdown(text):
#     # Remove markdown images
#     text = re.sub(r'!\[.*?\]\(https?://[^\s)]+\)', '', text)

#     # Replace markdown links [text](url) → keep text
#     text = re.sub(r'\[([^\]]+)\]\(https?://[^\s)]+\)', r'\1', text)

#     # Remove angle-bracket URLs
#     text = re.sub(r'<https?://[^>]+>', '', text)

#     # Remove inline raw URLs
#     text = re.sub(r'https?://\S+', '', text)

#     # Decode URL-encoded chars
#     text = unquote(text)

#     # Remove leading # from headings
#     text = re.sub(r'^\s*#+\s*', '', text, flags=re.MULTILINE)

#     # Remove bold (**text**) → text
#     text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)

#     # Remove italic (*text*) → text
#     text = re.sub(r'\*(.*?)\*', r'\1', text)

#     # Remove bullet * at line start
#     text = re.sub(r'^\s*\*\s+', '', text, flags=re.MULTILINE)

#     return text

In [42]:
import re
from urllib.parse import unquote

def clean_markdown_links(text):
    # Remove markdown images: ![alt](https://...)
    text = re.sub(r'!\[.*?\]\(https?://[^\s)]+\)', '', text)

    # Replace markdown links [text](https://...) → keep just the visible text
    text = re.sub(r'\[([^\]]+)\]\(https?://[^\s)]+\)', r'\1', text)

    # Remove angle-bracket links like <https://...>
    text = re.sub(r'<https?://[^>]+>', '', text)

    # Remove plain inline URLs like https://...
    text = re.sub(r'https?://\S+', '', text)

    # Decode URL-encoded characters
    text = unquote(text)

    return text

In [ ]:
# import os
# import psycopg2
# import json
# from dotenv import load_dotenv
# from langchain_pinecone import PineconeVectorStore
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain.schema import Document

# # Load environment variables
# load_dotenv()

# # Connect to PostgreSQL
# pg_conn = psycopg2.connect(
#     dbname="iplink4",
#     user="postgres",
#     password="CrappassFunTime25!",
#     host="iplink-4.c0nm66oygoar.us-east-1.rds.amazonaws.com",
#     port="5432"
# )
# pg_cursor = pg_conn.cursor()

# # Fetch rows
# pg_cursor.execute("SELECT id, title, contents FROM ip_new WHERE contents IS NOT NULL;")
# rows = pg_cursor.fetchall()

# # Set up embedding model and vector store
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# vector_store = PineconeVectorStore.from_existing_index(index_name="ip-link", embedding=embeddings)

# # Constants
# MAX_METADATA_BYTES = 40960  # 40KB metadata limit

# def prepare_document(text, metadata):
#     """
#     Returns a single Document with full text and metadata.
#     If metadata exceeds Pinecone's limit, logs and skips the document.
#     """
#     meta_str = json.dumps(metadata)
#     meta_bytes = len(meta_str.encode("utf-8"))

#     if meta_bytes > MAX_METADATA_BYTES:
#         print(f"⚠️ Skipping doc {metadata.get('source_id')} — metadata too large ({meta_bytes} bytes).")
#         return []

#     return [Document(page_content=text, metadata=metadata)]

# # Prepare all document entries
# docs = []
# for row in rows:
#     doc_id, title, contents = row
#     combined = f"{title.strip()}\n\n{contents.strip()}"
#     metadata = {"source_id": doc_id}
#     docs.extend(prepare_document(combined, metadata))

# # Final sanity check (optional): verify size if needed
# # (Skip this if you're not chunking)

# # Upload to Pinecone in batches
# def batched_upsert(docs, batch_size=50):
#     """Upload documents to Pinecone in safe batches."""
#     for i in range(0, len(docs), batch_size):
#         batch = docs[i:i + batch_size]
#         texts = [doc.page_content for doc in batch]
#         metadatas = [doc.metadata for doc in batch]
#         vector_store.add_texts(texts=texts, metadatas=metadatas)
#         print(f"✅ Uploaded batch {i // batch_size + 1} with {len(batch)} documents.")

# # Run upload
# batched_upsert(docs)
# print(f"🎉 Successfully uploaded {len(docs)} documents to Pinecone.")


# Independent Pinecone Upsert Script. Use this!!! -->

In [ ]:
import os
import psycopg2
import json
import uuid
from tqdm import tqdm
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone

# Load environment variables
load_dotenv()

# Initialize Pinecone client (v3 style)
pc = Pinecone(api_key="pcsk_5cxnbw_Km7NBkBocGHmR13BpxbiUkzLQ2sVmytwg5H1zZh3ekvs29WaREVobLmAe3KcdKM")
index = pc.Index(host="ip-link-6fzxslk.svc.aped-4627-b74a.pinecone.io")

# Connect to PostgreSQL
pg_conn = psycopg2.connect(
    dbname="iplink4",
    user="postgres",
    password="CrappassFunTime25!",
    host="iplink-4.c0nm66oygoar.us-east-1.rds.amazonaws.com",
    port="5432"
)
pg_cursor = pg_conn.cursor()

# Fetch data
pg_cursor.execute("SELECT id, title, contents FROM ip_new WHERE contents IS NOT NULL;")
rows = pg_cursor.fetchall()

# Embedding model
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Constants
BATCH_SIZE = 50
NAMESPACE = ""

# Prepare and upsert
vectors = []
for doc_id, title, contents in tqdm(rows):
    if not title or not contents:
        continue
    text = f"{title.strip()}\n\n{contents.strip()}"
    # text = clean_markdown_links(raw_text)
        
    metadata = {"source_id": doc_id, "title": title.strip()}
    embedding = embedder.embed_documents([text])[0]
    vectors.append((str(uuid.uuid4()), embedding, metadata))

# Batch upsert to Pinecone
for i in range(0, len(vectors), BATCH_SIZE):
    batch = vectors[i:i + BATCH_SIZE]
    index.upsert(vectors=batch, namespace=NAMESPACE)
    print(f"✅ Uploaded batch {i // BATCH_SIZE + 1} with {len(batch)} vectors.")

print(f"🎉 Done! Uploaded {len(vectors)} total documents to Pinecone.")


100%|██████████| 33644/33644 [59:29<00:00,  9.43it/s]  


✅ Uploaded batch 1 with 50 vectors.
✅ Uploaded batch 2 with 50 vectors.
✅ Uploaded batch 3 with 50 vectors.
✅ Uploaded batch 4 with 50 vectors.
✅ Uploaded batch 5 with 50 vectors.
✅ Uploaded batch 6 with 50 vectors.
✅ Uploaded batch 7 with 50 vectors.
✅ Uploaded batch 8 with 50 vectors.
✅ Uploaded batch 9 with 50 vectors.
✅ Uploaded batch 10 with 50 vectors.
✅ Uploaded batch 11 with 50 vectors.
✅ Uploaded batch 12 with 50 vectors.
✅ Uploaded batch 13 with 50 vectors.
✅ Uploaded batch 14 with 50 vectors.
✅ Uploaded batch 15 with 50 vectors.
✅ Uploaded batch 16 with 50 vectors.
✅ Uploaded batch 17 with 50 vectors.
✅ Uploaded batch 18 with 50 vectors.
✅ Uploaded batch 19 with 50 vectors.
✅ Uploaded batch 20 with 50 vectors.
✅ Uploaded batch 21 with 50 vectors.
✅ Uploaded batch 22 with 50 vectors.
✅ Uploaded batch 23 with 50 vectors.
✅ Uploaded batch 24 with 50 vectors.
✅ Uploaded batch 25 with 50 vectors.
✅ Uploaded batch 26 with 50 vectors.
✅ Uploaded batch 27 with 50 vectors.
✅ Uploaded

# List Existing Indices

In [41]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_5cxnbw_Km7NBkBocGHmR13BpxbiUkzLQ2sVmytwg5H1zZh3ekvs29WaREVobLmAe3KcdKM")

# List all indexes
indexes = pc.list_indexes()
print(indexes)


[{
    "name": "ip-link-clean-links",
    "metric": "cosine",
    "host": "ip-link-clean-links-6fzxslk.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 768,
    "deletion_protection": "disabled",
    "tags": null
}, {
    "name": "ip-link",
    "metric": "cosine",
    "host": "ip-link-6fzxslk.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 768,
    "deletion_protection": "disabled",
    "tags": null
}, {
    "name": "skb-index",
    "metric": "cosine",
    "host": "skb-index-6fzxslk.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
      

# Query (For Raw Pinecone index without any preprocessing)

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone

# Initialize Pinecone + Index
# pc = Pinecone(api_key="YOUR_API_KEY")
index = pc.Index(host="ip-link-6fzxslk.svc.aped-4627-b74a.pinecone.io")

# Set up embedding model
embedder = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cpu"}
)

# Your query
query_text = "Where can I learn about a technology that converts CO₂ into a useful chemical (formaldehyde) using a simple catalyst and ambient conditions"

# Convert query to embedding
query_vector = embedder.embed_query(query_text)

# Query Pinecone
results = index.query(
    vector=query_vector,
    top_k=10,
    include_metadata=True,
    namespace=""
)

# Print results
for match in results["matches"]:
    print(f"\n🔎 ID: {match['id']}")
    print(f"📈 Score: {match['score']:.4f}")
    print(f"📎 Metadata: {match['metadata']}")



🔎 ID: 40b56253-87bf-47f0-b0ce-16d68a03176e
📈 Score: 0.6833
📎 Metadata: {'source_id': 29402.0, 'title': 'Systems and Methods for Converting Carbon Dioxide into Chemical Feedstock'}

🔎 ID: e699ce6a-1799-4ba6-93c9-6d7397fe4cc2
📈 Score: 0.6779
📎 Metadata: {'source_id': 37563.0, 'title': 'Conversion of carbon dioxide to formaldehyde via a bis(silyl)acetal'}

🔎 ID: 33728e08-2aca-49ab-8664-73057512fb90
📈 Score: 0.6316
📎 Metadata: {'source_id': 19120.0, 'title': '18KU057L - Carbon Dioxide Conversion in Liquids Expanded by Carbon Dioxide'}

🔎 ID: f54eb2a2-f766-44a5-9343-38db4ecaf66b
📈 Score: 0.6268
📎 Metadata: {'source_id': 21215.0, 'title': 'Photocatalytic Composite for Carbon Dioxide Conversion to Methane'}

🔎 ID: f69546e2-ebfd-4d52-a7ca-e216220271de
📈 Score: 0.6138
📎 Metadata: {'source_id': 30343.0, 'title': 'Catalyst Technology for One-Step Upgrading of Biomass-Derived Chemicals'}

🔎 ID: b4e50099-d5e0-401b-be14-2697f5f111ba
📈 Score: 0.6084
📎 Metadata: {'source_id': 11634.0, 'title': 'Produ

# Query 2 (Query Pinecone index that preprocessed docs)

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone

# Initialize Pinecone + Index
# pc = Pinecone(api_key="YOUR_API_KEY")
index = pc.Index(host="https://ip-link-clean-links-6fzxslk.svc.aped-4627-b74a.pinecone.io")

# Set up embedding model
embedder = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cpu"}
)

# Your query
query_text = "Where can I learn about a technology that converts CO₂ into a useful chemical (formaldehyde) using a simple catalyst and ambient conditions"

# Convert query to embedding
query_vector = embedder.embed_query(query_text)

# Query Pinecone
results = index.query(
    vector=query_vector,
    top_k=10,
    include_metadata=True,
    namespace=""
)

# Print results
for match in results["matches"]:
    print(f"\n🔎 ID: {match['id']}")
    print(f"📈 Score: {match['score']:.4f}")
    print(f"📎 Metadata: {match['metadata']}")


🔎 ID: 57deab15-9298-4507-bb85-213de0bbe828
📈 Score: 0.6779
📎 Metadata: {'source_id': 37563.0, 'title': 'Conversion of carbon dioxide to formaldehyde via a bis(silyl)acetal'}

🔎 ID: f1806f5b-0daf-4e4b-8d97-bdea800ec4c1
📈 Score: 0.6652
📎 Metadata: {'source_id': 29402.0, 'title': 'Systems and Methods for Converting Carbon Dioxide into Chemical Feedstock'}

🔎 ID: a97749fc-de76-4446-8d4d-c3666dd4e408
📈 Score: 0.6270
📎 Metadata: {'source_id': 21215.0, 'title': 'Photocatalytic Composite for Carbon Dioxide Conversion to Methane'}

🔎 ID: 6d41476d-3187-438c-8680-4a0f43159041
📈 Score: 0.6228
📎 Metadata: {'source_id': 19120.0, 'title': '18KU057L - Carbon Dioxide Conversion in Liquids Expanded by Carbon Dioxide'}

🔎 ID: 05d83231-d1c8-4e26-a362-29991fde9fcf
📈 Score: 0.6138
📎 Metadata: {'source_id': 30343.0, 'title': 'Catalyst Technology for One-Step Upgrading of Biomass-Derived Chemicals'}

🔎 ID: 8c3fe9e0-7ec0-4b0d-9fb8-00f30eb891e8
📈 Score: 0.6094
📎 Metadata: {'source_id': 15724.0, 'title': '11-43

# DELETE FROM INDEX  

In [ ]:
from pinecone import Pinecone

# Initialize client (use your real API key)
pc = Pinecone(api_key="pcsk_5cxnbw_Km7NBkBocGHmR13BpxbiUkzLQ2sVmytwg5H1zZh3ekvs29WaREVobLmAe3KcdKM")
index = pc.Index(host="https://ip-link-6fzxslk.svc.aped-4627-b74a.pinecone.io")

# Delete everything in the index
index.delete(delete_all=True, namespace="")

print(f"All vectors deleted from {index}.")

🗑️ All vectors deleted from ip-link2.


# Using the preprocessed(markdown) index as it performed better overall.